In [ ]:
pip install pmdarima

     |████████████████████████████████| 1.4 MB 7.7 MB/s 
     |████████████████████████████████| 9.8 MB 44.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import os
import matplotlib.pyplot as plt
import time
plt.style.use(style='seaborn')

In [ ]:
df=pd.read_csv('/content/IAM - Données Historiques (1).csv')

In [ ]:
df.head(5)

,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %
0,30/08/2021,"140,70","141,35","141,35","140,60","4,12K","-0,21%"
1,27/08/2021,"141,00","140,00","141,00","140,00","90,83K","0,79%"
2,26/08/2021,"139,90","139,25","140,00","139,25","93,64K","0,29%"
3,25/08/2021,"139,50","140,40","140,70","139,50","17,01K","-0,78%"
4,24/08/2021,"140,60","140,25","140,90","140,00","3,61K","-0,21%"


In [ ]:
df['Date'] = pd.to_datetime(df.Date,format='%d/%m/%Y')
df.index = df['Date']

In [ ]:
df=df[["Dernier"]].copy()
df1=df.replace('[^\d].','',regex=True).astype(float)

In [ ]:
df1.describe()

,Dernier
count,1234.000000
mean,1408.525122
std,65.366679
min,1125.000000
25%,1370.000000
50%,1400.000000
75%,1450.000000
max,1580.000000


In [ ]:
from statsmodels.tsa.stattools import adfuller
result=adfuller(df1.Dernier.dropna())
print(f"ADF Statistic:{result[0]}")
print(f"p-value:{result[1]}")

ADF Statistic:-3.3109881685835387
p-value:0.01439439111334547


In [ ]:
from pmdarima.arima.utils import ndiffs

In [ ]:
ndiffs(df1.Dernier,test="adf")

1

In [ ]:
#train test split
n=int(len(df1)*0.8)
train=df1.Dernier[:n]
test=df1.Dernier[n:]

In [ ]:
print(len(train))
print(len(test))

993
249


In [ ]:
model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=8029.416, Time=1.59 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=8039.570, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=8041.376, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=8041.383, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=8037.573, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=8045.044, Time=0.68 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=8045.115, Time=0.53 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=8031.132, Time=2.05 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=8031.117, Time=2.16 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=8043.316, Time=0.87 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=8042.557, Time=1.10 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=8042.643, Time=1.51 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=2.76 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=8027.419, Time=0.99 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

In [ ]:
# Determine model parameters
model.fit(train)
order = model.get_params()['order']
print('ARIMA order:', order, '\n')

ARIMA order: (2, 1, 2) 



In [ ]:
from statsmodels.tsa.arima_model import ARIMA


In [ ]:
forceast=model.predict(len(test))

In [ ]:
plt.figure()
plt.plot(test,label="actuelle")
plt.plot(train,label="train")
plt.plot(forceast,label="prediction")
plt.title("Prediction vs actuelle")
plt.xlabel('Date')
plt.ylabel('Dernier prix ')
plt.legend(loc='upper left')
plt.show()

ValueError: ignored

<Figure size 576x396 with 1 Axes>